In [6]:
# loading libraries
import pandas as pd
from IPython.display import display, Math, Latex

# Regresion logistica

* Basada en la probabilidad condicionada modelo de clasificacion
* variante de la regresion lineal, la variable de salida binaria o catégirica.
* necesita una relacion lineal entre predictores y variables de salida con alguna trasformacion

$y = \frac{1}{1+e^{-(\alpha +\beta x)}}$

In [7]:
#cargar dataset
purchase = pd.read_csv("Gender Purchase.csv")

In [8]:
purchase.head(2)

,Gender,Purchase
0,Female,Yes
1,Female,Yes


In [9]:
cross_tab = pd.crosstab(purchase['Gender'],purchase['Purchase'])

In [10]:
cross_tab

Purchase,No,Yes
Gender,,
Female,106,159
Male,125,121


In [11]:
cross_tab.astype('float').div(cross_tab.sum(axis = 1),axis=0)

Purchase,No,Yes
Gender,,
Female,0.40000,0.60000
Male,0.50813,0.49187


#### ¿cuál es la probabilidad de que un cliente compre un producto dado que es un hombre?

In [12]:
display(Math(r'P(Compre \ y\ es \ hombre | es \ hombre) = \frac {121}{246}'))

<IPython.core.display.Math object>

In [13]:
# Teniendo en cuenta los resultados de la tabla 
# de contingencia
(121/511)/(246/511)

0.491869918699187

#### cual es la probabilidad de que un cliente sea mujer dado que compró

In [15]:
display(Math(r'P(es \ mujer \ y\ compra | compra) = \frac {159}{159 + 121}'))

<IPython.core.display.Math object>

#### cual es la probabilidad de que un cliente sea hombre dado que compró

In [16]:
display(Math(r'P(es \ hombre | compra) = \frac {121}{159 + 121}'))

<IPython.core.display.Math object>

In [17]:
display(Math(r'P(no \ compra | es \ hombre) = \frac {125}{125 + 121}'))

<IPython.core.display.Math object>

In [18]:
display(Math(r'P(no \ compra | es \ mujer) = \frac {106}{106 + 159}'))

<IPython.core.display.Math object>

## Ratio de probabilidades
cocientes entre casos de exito sobre los de fracaso en el suceso estudiado y para cada grupo, va desde cero hasta infinito

In [19]:
display(Math(r'P_f = Probabilidad\ de\ compras\ hechas\ por\ mujeres'))
display(Math(r'Odds_{Purchase\ Female} = \frac{P_f}{1-P_f}'))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [20]:
display(Math(r'P_m = Probabilidad\ de\ compras\ hechas\ por\ hombres'))
display(Math(r'Odds_{Purchase\ male} = \frac{P_m}{1-P_m}'))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Implementación método maxima verosimilitud para la regresión logística

In [ ]:
from IPython.display import display, Math, Latex

#### Función de entorno

In [ ]:
display(Math(r'L(\beta) = \prod_{i=1}^n P_i^{y_i}*(1-P_i)^{y_i}'))

<IPython.core.display.Math object>

In [ ]:
def likelihood(y,pi):
    import numpy as np
    total_sum = 1
    sum_in = list(range(1, len(y)+1))
    for i in range(len(y)):
        sum_in[i] = np.where(y[i]==1,pi[i],1-pi[i])
        total_sum = total_sum * sum_in[i]
    return total_sum
    

#### Calcular probabilidades para cada observación

In [ ]:
display(Math(r'P_i = P(x_i)=\frac{1}{1+e^{-\sum_{i=1}^n beta_j\cdot x_ij}}'))

<IPython.core.display.Math object>

In [ ]:
def logitprobs(X, beta):
    import numpy as np
    
    n_rows = np.shape(X)[0]
    n_cols = np.shape(X)[1]
    pi =list(range(1,n_rows+1))
    expon = list(range(1,n_rows+1))
    for i in range(n_rows):
        expon[i] = 0
        for j in range(n_cols):
            ex = beta[j]*X[i][j]
            expon[i] = ex + expon[i]
        with np.errstate(divide='ignore', invalid='ignore'):
            pi[i]= 1/(1+np.exp(-expon[i]))
    return pi

#### Calcular la matriz diagonal

In [ ]:
display(Math(r'W= diag(P_i \cdot (1-p_i))_{i=1}^n'))

<IPython.core.display.Math object>

In [ ]:
def findw(pi):
    import numpy as np
    n = len(pi)
    W = np.zeros(n*n).reshape(n,n)
    for i in range(n):
        print(i)
        W[i][i] = pi[i]*(1-pi[i])
        W[i][i].astype(float)
    return W

#### Obtener la solución de la función logistica

In [ ]:
display(Math(r"\beta_{n+1} = \beta_n - \frac{f(x_n)}{f'(x_n)}"))
display(Math(r"f(x_n) = X(Y - P)"))
display(Math(r"f'(x_n) = XWX^T"))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [ ]:
def logistics(X, Y, limit):
    import numpy as np
    from numpy import linalg
    nrow = np.shape(X)[0]
    bias = np.ones(nrow).reshape(nrow,1)
    X_new = np.append(X, bias, axis = 1)
    ncol = np.shape(X_new)[1]
    beta = np.zeros(ncol).reshape(ncol,1)
    root_dif = np.array(range(1,ncol+1)).reshape(ncol,1)
    iter_i = 10000
    while(iter_i>limit):
        print("Iter:i"+str(iter_i) + ", limit:" + str(limit))
        pi = logitprobs(X_new, beta)
        print("Pi:"+str(pi))
        W = findw(pi)
        print("W:"+str(W))
        num = (np.transpose(np.matrix(X_new))*np.matrix(Y - np.transpose(pi)).transpose())
        den = (np.matrix(np.transpose(X_new))*np.matrix(W)*np.matrix(X_new))
        root_dif = np.array(linalg.inv(den)*num)
        beta = beta + root_dif
        print("Beta: "+str(beta))
        iter_i = np.sum(root_dif*root_dif)
        ll = likelihood(Y, pi)
    return beta

Comprobación experimental

In [ ]:
import numpy as np

In [ ]:
X = np.array(range(10)).reshape(10,1)

In [ ]:
y = [0,0,0,0,1,0,1,0,1,1]

In [ ]:
bias = np.ones(10).reshape(10,1)
X_new = np.append(X,bias,axis=1)

In [ ]:
X_new

array([[0., 1.],
       [1., 1.],
       [2., 1.],
       [3., 1.],
       [4., 1.],
       [5., 1.],
       [6., 1.],
       [7., 1.],
       [8., 1.],
       [9., 1.]])

In [ ]:
logistics(X,y,0.00001)

Iter:i10000, limit:1e-05
Pi:[array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5])]
0
1
2
3
4
5
6
7
8
9
W:[[0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.25 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.25 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.25 0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.25 0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.25 0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.25 0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.25 0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.25]]
Beta: [[ 0.43636364]
 [-2.36363636]]
[0.5][0.5][0.5]
totsum[0.5]
[0.5][0.5][0.5]
totsum[0.25]
[0.5][0.5][0.5]
totsum[0.125]
[0.5][0.5][0.5]
totsum[0.0625]
[0.5][0.5][0.5]
totsum[0.03125]
[0.5][0.5][0.5]
totsum[0.015625]
[0.5][0.5][0.5]
totsum[0.0078125]
[0.5][0.5][0.5]
totsum[0

array([[ 0.66220827],
       [-3.69557172]])

In [ ]:
import statsmodels.api as sm

In [ ]:
logit_model = sm.Logit(y,X_new)
resultados = logit_model.fit()

Optimization terminated successfully.
         Current function value: 0.431012
         Iterations 6


In [ ]:
resultados.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                        Results: Logit
===============================================================
Model:              Logit            Pseudo R-squared: 0.360   
Dependent Variable: y                AIC:              12.6202 
Date:               2021-11-08 16:23 BIC:              13.2254 
No. Observations:   10               Log-Likelihood:   -4.3101 
Df Model:           1                LL-Null:          -6.7301 
Df Residuals:       8                LLR p-value:      0.027807
Converged:          1.0000           Scale:            1.0000  
No. Iterations:     6.0000                                     
-----------------------------------------------------------------
          Coef.    Std.Err.      z      P>|z|     [0.025   0.975]
-----------------------------------------------------------------
x1        0.6622     0.4001    1.6551   0.0979   -0.1220   1.4464
const    -3.6956     2.2889   -1.6145   0.1064   -8.1818   0.7906
===============================================================

"""

# Implementación regresión logística

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/ColabNotebooks/ML_udemy/bank.csv",sep=';')

In [ ]:
data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,30,blue-collar,married,basic.9y,no,yes,no,cellular,may,fri,487,2,999,0,nonexistent,-1.8,92.893,-46.2,1.313,5099.1,no
1,39,services,single,high.school,no,no,no,telephone,may,fri,346,4,999,0,nonexistent,1.1,93.994,-36.4,4.855,5191.0,no
2,25,services,married,high.school,no,yes,no,telephone,jun,wed,227,1,999,0,nonexistent,1.4,94.465,-41.8,4.962,5228.1,no
3,38,services,married,basic.9y,no,unknown,unknown,telephone,jun,fri,17,3,999,0,nonexistent,1.4,94.465,-41.8,4.959,5228.1,no
4,47,admin.,married,university.degree,no,yes,no,cellular,nov,mon,58,1,999,0,nonexistent,-0.1,93.200,-42.0,4.191,5195.8,no


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4119 entries, 0 to 4118
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             4119 non-null   int64  
 1   job             4119 non-null   object 
 2   marital         4119 non-null   object 
 3   education       4119 non-null   object 
 4   default         4119 non-null   object 
 5   housing         4119 non-null   object 
 6   loan            4119 non-null   object 
 7   contact         4119 non-null   object 
 8   month           4119 non-null   object 
 9   day_of_week     4119 non-null   object 
 10  duration        4119 non-null   int64  
 11  campaign        4119 non-null   int64  
 12  pdays           4119 non-null   int64  
 13  previous        4119 non-null   int64  
 14  poutcome        4119 non-null   object 
 15  emp.var.rate    4119 non-null   float64
 16  cons.price.idx  4119 non-null   float64
 17  cons.conf.idx   4119 non-null   f